In [1]:
import numpy as np
import pandas as pd

### Data

In [2]:
import sklearn
from ucimlrepo import fetch_ucirepo

In [20]:
npha = fetch_ucirepo(id = 936) 

feature_data = npha.data.features
target_data = npha.data.targets

In [48]:
dataset_name = 'nursery'
target_name = target_data.columns[0]
feature_names = list(feature_data.columns)

In [49]:
X = feature_data.to_numpy()
y = target_data.to_numpy()

In [50]:
feature_encoder = sklearn.preprocessing.OrdinalEncoder()
feature_encoder.fit(X)

target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(y.squeeze())

X = feature_encoder.transform(X)
y = target_encoder.transform(y)

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [51]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y.squeeze(), test_size = 0.2, random_state = 51)

### Model

In [52]:
from tensorflow import keras

In [53]:
def build_model(input_shape, classes, n_layers = 5, n_nodes_l = 2, n_nodes_u = 10, optimizer = 'adam', learning_rate = 0.001):

    '''
    Dynamically creates fully-connected neural network given hyperparameters.
    '''
    
    model = keras.Sequential()
    model.add(keras.Input(shape = (input_shape,)))
    for i in range(n_layers):
        model.add(keras.layers.Dense(np.random.choice([i for i in range(n_nodes_l, n_nodes_u)]), activation = np.random.choice(['relu'])))
    model.add(keras.layers.Dense(classes, activation = 'softmax'))

    if optimizer == 'sgd':
        optim = keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'adam':
        optim = keras.optimizers.Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [58]:
n_classes = len(np.unique(y_train))
input_shape = len(X_train[0])
n_layers = 5
n_nodes_l = 5
n_nodes_u = 30
optimizer = 'adam'
learning_rate = 0.1
epochs = 25
batch_size = 1

In [ ]:
model = build_model(
    input_shape,
    n_classes,
    n_layers = n_layers, 
    n_nodes_l = n_nodes_l,
    n_nodes_u = n_nodes_u, 
    optimizer = optimizer, 
    learning_rate = learning_rate
)

model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.2)
model.summary()

Epoch 1/25


In [12]:
y_pred = model(X_test)
y_pred = np.argmax(y_pred, axis = 1)

mlp_accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {mlp_accuracy:.4f}")

Test Accuracy: 0.9115


#### Explanations

In [13]:
import os
import re
import shap
import json
from modules.method import Method
from lime.lime_tabular import LimeTabularExplainer
from modules.metrics import markov_blanket, structure_metrics, distribution_drift, average_distribution_drift, divergence_plot, fidelity_plot

In [14]:
def lime_attribution(instance, training_data, model, feature_names, target_name):

    explainer = LimeTabularExplainer(
        training_data = training_data,
        feature_names = feature_names,
        class_names = target_name,
        mode = 'classification'
    )

    exp = explainer.explain_instance(
        data_row = instance, 
        predict_fn = model.predict, 
        num_features = len(feature_names)
    )

    important_features = exp.as_list()
    
    sorted_features = [re.sub(r"[\d+\*/<>.=() ]", "", feature) for feature, weight in sorted(important_features, key = lambda x: abs(x[1]), reverse = True)]

    return sorted_features

In [15]:
def shap_attribution(instance, training_data, model, feature_names):

    explainer = shap.Explainer(
        model.predict,
        training_data
    )

    shap_values = explainer(instance.reshape(1, -1))

    pred_class = np.argmax(model.predict(instance.reshape(1, -1)))
    feature_shap_values = shap_values.values[0, :, pred_class]

    important_features = list(zip(feature_names, feature_shap_values))
    print(important_features)

    sorted_features = [re.sub(r"[^a-zA-Z-]+", "", feature) for feature, weight in sorted(important_features, key = lambda x: abs(x[1]), reverse = True)]

    return sorted_features

In [18]:
results = []
experiment_name = os.path.join("./data/results", "big t")
n_trials = 10

In [19]:
for _ in range(100):
    
    method = Method(
        model = model, 
        training_data = X_train, 
        feature_names = feature_names, 
        target_name = target_name, 
        n_samples = 5000, 
        rep_prob = 0.5
    )

    instance_idx = np.random.randint(X_test.shape[0])
    instance = X_test[instance_idx]
    
    method.log_data(instance)
    method.get_structure()

    no_features = []
    method_features = list(method.blanket)
    lime_features = lime_attribution(instance, X_train, model, feature_names, target_name)[:len(method_features)]
    shap_features = shap_attribution(instance, X_train, model, feature_names)[:len(method_features)]

    no_new, no_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = no_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    method_new, method_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = method_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )

    lime_new, lime_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = lime_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )

    shap_new, shap_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = shap_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    no_avg, no_values = average_distribution_drift(no_og, no_new)
    method_avg, method_values = average_distribution_drift(method_og, method_new)
    lime_avg, lime_values = average_distribution_drift(lime_og, lime_new)
    shap_avg, shap_values = average_distribution_drift(shap_og, shap_new)
    
    js_values = np.stack((no_values, method_values, lime_values, shap_values))
    
    print(f'= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =')
    print(f'No Features Divergence: {no_avg}')
    print(f'Method Features Divergence: {method_avg}')
    print(f'LIME Features Divergence: {lime_avg}')
    print(f'Shap Featues Divergence: {shap_avg}')

    results.append({

        "dataset": dataset_name,

        "target": target_name,
        "features": feature_names,
        "mlp_accuracy": float(mlp_accuracy),
        
        "method_features": method_features,
        "lime_features": lime_features,
        "shap_features": shap_features,
        
        "no_new": no_new.tolist(),
        "method_new": method_new.tolist(),
        "lime_new": lime_new.tolist(),
        "shap_new": shap_new.tolist(),
        
        "no_og": no_og.tolist(),
        "method_og": method_og.tolist(),
        "lime_og": lime_og.tolist(),
        "shap_og": shap_og.tolist(),
        
        "no_values": list(no_values),
        "method_values": list(method_values),
        "lime_values": list(lime_values),
        "shap_values": list(shap_values),
        
        "no_avg": float(no_avg),
        "method_avg": float(method_avg),
        "lime_avg": float(lime_avg),
        "shap_avg": float(shap_avg),
    })

    with open(f'{experiment_name}.json', 'w') as f:

        json.dump(results, f, indent = 4)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-left-square': 'C', 'bottom-middle-square': 'C', 'bottom-right-square': 'C', 'class': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-left-square': 'C', 'bottom-middle-square': 'C', 'bottom-right-square': 'C', 'class': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-l

  0%|          | 0/1000000 [00:00<?, ?it/s]

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step
1046/1046 ━━━━━━━━━━━━━━━━━━━━ 1s 497us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[('top-left-square', np.float64(-0.015944096777090726)), ('top-middle-square', np.float64(0.06568228589344206)), ('top-right-square', np.float64(0.06692584291917696)), ('middle-left-square', np.float64(0.27425194546829346)), ('middle-middle-square', np.float64(-0.10874979428390928)), ('middle-right-square', np.float64(0.11848490525163319)), ('bottom-left-square', np.float64(0.10242230457817235)), ('bottom-middle-square', np.float64(-0.026452236452736873)), ('bottom-right-square', np.float64(0.1695853628404412))]
= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =
No Features Divergence: 0.4852023838613736
Method Features Divergence: 2.389626756913685e-16
LIME Features Divergence: nan
Shap Featues Divergence: 2.389626756913685e-16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step


/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/scipy/spatial/distance.py:1388: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-left-square': 'C', 'bottom-middle-square': 'C', 'bottom-right-square': 'C', 'class': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-left-square': 'C', 'bottom-middle-square': 'C', 'bottom-right-square': 'C', 'class': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 

...

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 1s 493us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[('top-left-square', np.float64(0.05813978553674644)), ('top-middle-square', np.float64(-0.002594254109355234)), ('top-right-square', np.float64(0.2323603871387734)), ('middle-left-square', np.float64(0.011633492193152566)), ('middle-middle-square', np.float64(-0.15037833676461262)), ('middle-right-square', np.float64(0.04602470942805106)), ('bottom-left-square', np.float64(0.023598660171427595)), ('bottom-middle-square', np.float64(0.09145556104103743)), ('bottom-right-square', np.float64(0.04355341493708347))]
= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =
No Features Divergence: 0.20794418381380503
Method Features Divergence: 3.815483532825526e-06
LIME Features Divergence: 4.083969523180744e-12
Shap Featues Divergence: 4.083969523180744e-12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-left-square': 'C', 'bottom-middle-square': 'C', 'bottom-right-square': 'C', 'class': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-left-square': 'C', 'bottom-middle-square': 'C', 'bottom-right-square': 'C', 'class': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'top-left-square': 'C', 'top-middle-square': 'C', 'top-right-square': 'C', 'middle-left-square': 'C', 'middle-middle-square': 'C', 'middle-right-square': 'C', 'bottom-l

  0%|          | 0/1000000 [00:00<?, ?it/s]

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step
970/970 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[('top-left-square', np.float64(-0.16479543630864044)), ('top-middle-square', np.float64(0.06164116945677724)), ('top-right-square', np.float64(-0.006976804842766336)), ('middle-left-square', np.float64(-0.082730886316087)), ('middle-middle-square', np.float64(-0.08250955667224019)), ('middle-right-square', np.float64(0.07009704490547124)), ('bottom-left-square', np.float64(0.2319134490105228)), ('bottom-middle-square', np.float64(0.16220076660110644)), ('bottom-right-square', np.float64(0.164953673738161))]
= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =
No Features Divergence: 0.20794418391238317
Method Features Divergence: 0.12334410518925706
LIME Features Divergence: nan
Shap Featues Divergence: 0.10251386185870948


/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/scipy/spatial/distance.py:1388: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)
